# 1. Information about the submission

## 1.1 Name and number of the assignment 

## **Word Sense Induction(Knowledge-free)**, Assignment 1.

## 1.2 Student name

## **Albert Sayapin**

## 1.3 Codalab user ID

## **albertSayapin**

## 1.4 Additional comments

## *This is a very cool task:)*

Checked it on **Google Colab** successfully!

# 2. Technical Report

## 2.1 Methodology 

The main problem I tried to solve is **Word Sense Induction**, meaning that we want to automatically discover the senses of semantically ambiguous words from unannotated text.

Generally speaking, I used the model which consists of 3 components(Knowledge-Free):
- precalculated words embeddings(to capture word semantics);
- particular weighting scheme of embeddings(to approximate context semantics);
- 1 step / 2 step clustering technique, namely how many clustering algorithms I use to get better results;

Frankly, I was highly inspired by:
- http://www.dialog-21.ru/media/4535/kutuzovab.pdf
- http://www.dialog-21.ru/media/4538/arefyevn_ermolaevp_panchenkoa.pdf

There are some essential steps in this project I had to walk through:

1. **Data preprocessing**: I had to preprocess "context" column of every dataset(train/test):
- *Lemmatized* all the words by pymystem3.Mystem stemmer and made them lowercase(Normalization step);
- *Dropped* all the words from nltk *Russian stopwords* list(As they do not bring any additional information);
- *Eliminated* all the words with *length* less than 3 and target word("word" column) as well;
- *Calculated words occurrences* both locally(for every context for every word) and globally(for every word) and left only unique tokens;
(It could help to get more elaborate weighting method)

2. **Model training**: I had to find optimal parameters for my model(using Brute Force as the size of the problem is not that big):
- *pretrained word embeddings*: 
    - geowac_lemmas_none_fasttextskipgram_300_5_2020
    - ruscorpora_upos_skipgram_300_5_2018
- *weighting scheme*: Average, Sum, Local Sum;
- *Normalize* context vectors or not?
- *1 step or 2 step clustering*? (Used AffinityPropagation for the first step to identify the number of clusters)
- What *clustering algorithm* to use? (AgglomerativeClustering, KMeans, SpectralClustering)

3. **Model evaluation**: I had to use *Adjusted Rank Score* to compare different clusterings.

4. **Send the results**: the test.csv -> .zip files to CodaLab system.

*Some words about why I decided to move this particular way:*

*First of all*, I used fastText Skipgram model that was precalculated on GeoWAC data. It is a good choice to get results quickly and with high accuracy(especially Gensim interface). 

*Secondly*, in order to represent context out of words you have to combine them in some way to get one representation.
It is not pretty intuitive what method to use: just take the average or sum them up, weight all the words according to their relative frequency. Hence, I decided to look at them all and choose the best based on train ARS results.

*Thirdly*, It is clustering of course. The challenging part I think, as we do not know how to define the number of clusters beforehand. Thanks to Affinity Propagation algorithm, it can find it almost automatically. After that we can just leave it as it is or use the next clustering algorithm that needs the number of clusters.

## 2.2 Discussion of results

### **Summary of the experiment:**
Here below you can the best results I achieved with the help of two models:
- **WSI18** -> which is based on *ruscorpora_upos_skipgram_300_5_2018*
- **WSI20** -> which is based on *geowac_lemmas_none_fasttextskipgram_300_5_2020*

### *Train.csv results:*

Method | ARS_wiki-wiki | ARS_bts-rnc | ARS_active-dict|
--- | --- | --- | --- |
WSI18 | 0.796 | 0.127 | 0.241 |
WSI20 | 0.819 | 0.144 | 0.303 |


### *Test.csv results:*

Method | ARS_wiki-wiki | ARS_bts-rnc | ARS_active-dict|
--- | --- | --- | --- |
Baseline(CodaLab) | 0.628 | 0.262 | 0.176 |
WSI18 | 1.0 | 0.153 | 0.116 |
WSI20 | 1.0 | 0.20 | 0.193 |


### *Optimal parameters for the models:*
Method | ARS_wiki-wiki | ARS_bts-rnc | ARS_active-dict|
--- | --- | --- | --- |
WSI18 | EM=S; N=1; C=AC; S=2; P=-3; | EM=S; N=1; C=AC; S=2; P=-2; | EM=W; N=1; C=KM; S=2; P=-1; |
WSI20 | EM=S; N=1; C=AC; S=2; P=-3; | EM=W; N=1; C=AC; S=2; P=-2; | EM=S; N=1; C=SC; S=2; P=-1; |

- EM = Embedding Method: {S: Sum; M: Mean; W: Weighted}
- N = Normalization: {0: False; 1: True}
- C = Clustering: {AC: AgglomerativeClustering; KM: KMeans; SC: SpectralClustering}
- S = Number of steps: {1, 2}
- P = Preference parameter for AffinityPropagation algorithm

The tables show us that the model based on *geowac_lemmas_none_fasttextskipgram_300_5_2020* works better on every dataset with not so big differences in other parameters.
Besides the method managed to beat the baseline on two datasets, excluding bts-rnc.
The problem may be with chosen embeddings, as they could not capture the most valuable signal from provided contexts.
That is why we can conclude that embeddings are a key component if we prefer to use this methodology.

### **Conclusion:**
As we can see from the results a model which is based on pretrained word embeddings can work pretty well.

However the success and prediction accuracy depends highly on embeddings quality and applicability, weighting scheme and clustering method. 

That is why the more we improve these components the more accurate this particular model can become.

# 3. Code

## 3.1 Requirements

In [1]:
# Some essential packages:
!pip install pymystem3==0.1.10
!pip install gensim==4.1.2
!pip install sklearn
!pip install pandas
!pip install nltk
!pip install wget
!pip install gdown

# Embeddings for the model:
!wget http://vectors.nlpl.eu/repository/20/214.zip
!unzip -o 214.zip -d ru_fasttext_model
!rm 214.zip

!wget http://vectors.nlpl.eu/repository/20/213.zip
!unzip -o 213.zip -d new_model
!rm 213.zip

!mkdir results


--2021-11-27 22:23:33--  http://vectors.nlpl.eu/repository/20/214.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920218982 (1,8G) [application/zip]
Saving to: ‘214.zip’

214.zip             100%[===================>]   1,79G  33,2MB/s    in 72s     

2021-11-27 22:24:45 (25,6 MB/s) - ‘214.zip’ saved [1920218982/1920218982]

Archive:  214.zip
  inflating: ru_fasttext_model/meta.json  
  inflating: ru_fasttext_model/model.model  
  inflating: ru_fasttext_model/model.model.vectors_ngrams.npy  
  inflating: ru_fasttext_model/model.model.vectors.npy  
  inflating: ru_fasttext_model/model.model.vectors_vocab.npy  
  inflating: ru_fasttext_model/README  
--2021-11-27 22:24:57--  http://vectors.nlpl.eu/repository/20/213.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.2

## 3.2 Download the data:

In [2]:
# To get train/test.csv:
!git clone https://github.com/nlpub/russe-wsi-kit.git
!gdown --id 11nFGL1EJEgCr6VBVorhe5fnn3hnWd38r
!mv wiki_test.csv russe-wsi-kit/data/main/wiki-wiki/wiki_test.csv


Cloning into 'russe-wsi-kit'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 148 (delta 4), reused 22 (delta 4), pack-reused 116
Receiving objects: 100% (148/148), 3.83 MiB | 4.70 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Downloading...
From: https://drive.google.com/uc?id=11nFGL1EJEgCr6VBVorhe5fnn3hnWd38r
To: /home/albert/skoltech_classes/term2/snlp/project/Statistical_Natural_Language_Processing/HW1/wiki_test.csv
100%|████████████████████████████████████████| 506k/506k [00:00<00:00, 4.73MB/s]


## 3.3 Data Preprocessing: 

In [3]:
import re
from collections import Counter

import pandas as pd
import nltk
from pymystem3 import Mystem

from nltk.corpus import stopwords

nltk.download("stopwords")
russian_stopwords = stopwords.words("russian")


def lemmatized_context(row, stemmer):
    s = row['context']
    target = row['word']
    
    tokens = stemmer.lemmatize(s.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
                and token != " " \
                and re.match('[\w\-]+$', token)\
                and (len(token) > 2)\
                and token != target]
    return tokens

def get_counter(row):
    s = row['context']
    c = Counter(s)
    return list(set(s)), c

def get_counter_global(data):
    return data.groupby("word").apply(lambda x: Counter(x["context"].sum()))

def read_preprocess(data_path):
    
    # Read the data:
    data = pd.read_csv(data_path, sep='\t')

    # Essential columns:
    cols = [
        'context_id', 'word', 'gold_sense_id',
        'predict_sense_id', 'context',
    ]

    # Leave only essential columns:
    data = data[cols]
    
    # Lemmatization:
    stemmer = Mystem()
    data['context'] = data.apply(lemmatized_context, 1, stemmer=stemmer)

    # Leave unique and count all:
    data["embedding_need"] = data.apply(get_counter, axis=1)

    # Calculate frequency based on all contexts:
    word2glob_freq = get_counter_global(data).to_dict()

    return data, word2glob_freq

[nltk_data] Downloading package stopwords to /home/albert/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Define necessary paths:
path = "russe-wsi-kit/data/main/"

wiki = "wiki-wiki/"
bts = "bts-rnc/"
actd = "active-dict/"

train_file = "train.csv"
test_file = "test.csv"


# Prepare the data:
wiki_train, wiki_train_global = read_preprocess(path + wiki + train_file)
wiki_test, wiki_test_global = read_preprocess(path + wiki + 'wiki_test.csv')

bts_train, bts_train_global = read_preprocess(path + bts + train_file)
bts_test, bts_test_global = read_preprocess(path + bts + test_file)

actd_train, actd_train_global = read_preprocess(path + actd + train_file)
actd_test, actd_test_global = read_preprocess(path + actd + test_file)

wiki_train.head()

,context_id,word,gold_sense_id,predict_sense_id,context,embedding_need
0,1,замок,1,NaN,"[владимир, мономах, любеч, многочисленный, укр...","([помимо, вышгородский, это, укреплять, также,..."
1,2,замок,1,NaN,"[шильонский, шильйон, известный, русскоязычный...","([располагать, представлять, известный, литера..."
2,3,замок,1,NaN,"[проведение, архитектурный, археологический, р...","([потти, год, кальюнди, наука, называться, арх..."
3,4,замок,1,NaN,"[топь, белокуров, легенда, завещание, мавра, ю...","([янушкевич, рождение, легенда, белокуров, ден..."
4,5,замок,1,NaN,"[великий, князь, литовский, гедимин, успешный,...","([год, второй, трок, правило, кейстут, литовск..."


## 3.4 My method of text processing

In [5]:
import operator

import numpy as np
from sklearn.cluster import(
    AgglomerativeClustering,
    AffinityPropagation,
    KMeans,
    SpectralClustering,
)
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt

import gensim

import warnings
warnings.filterwarnings('ignore')


def give_emb_local_inv(model, context, counter, size, norm=False):
    """     
        Calculates the context vector based on word embeddings and
        each word is weighted by its relative frequency within one context. 
        
        Parameters:
            - model: dictionary like structure, gives embedding by the word: model[word];
            - context: list of words defining the context;
            - counter: dictionary like structure, gives the number of times 
                a word was seen in this context by the key: model[key];
            - size: size of word embeddings;
            - norm: True - normalize, False - do not;
        
        Result: array
    """

    if len(context) == 0:
        return np.zeros(size)

    weights = np.array([1 / counter[word] for word in context])
    ov = np.average(model[context], axis=0, weights=weights)
    if norm: 
        ov /= np.linalg.norm(ov)
    return ov


def give_emb_average(model, context, counter, size, norm=False):
    """     
        Calculates the context vector based on average of word embeddings.

        Parameters:
            - model: dictionary like structure, gives embedding by the word: model[word];
            - context: list of words defining the context;
            - counter: dictionary like structure, gives the number of times 
                a word was seen in this context by the key: model[key];
            - size: size of word embeddings;
            - norm: True - normalize, False - do not;
        
        Result: array  
    """

    if len(context) == 0:
        return np.zeros(size)

    ov = model[context].mean(axis=0)
    if norm: 
        ov /= np.linalg.norm(ov)
    return ov


def give_emb_sum(model, context, counter, size, norm=False):
    """     
        Calculates the context vector based on sum of word embeddings.

        Parameters:
            - model: dictionary like structure, gives embedding by the word: model[word];
            - context: list of words defining the context;
            - counter: dictionary like structure, gives the number of times 
                a word was seen in this context by the key: model[key];
            - size: size of word embeddings;
            - norm: True - normalize, False - do not;
        
        Result: array  
    """

    if len(context) == 0:
        return np.zeros(size)
        
    ov = model[context].sum(axis=0)
    if norm: 
        ov /= np.linalg.norm(ov)
    return ov


def get_centroids(embeddings, c_labels, data):
    """     
        Calculates the centroids for particular word and labels.
        (input is projected on particular word])

        Parameters:
            - embeddings: particular word context embeddings;
            - c_labels: particular word context labels;
            - data: Pandas Dataframe that has necessary columns:
                "word", "context;
        
        Result: dictionary: {label:(embedding, list of words for the context)}  
    """

    centroids = {}
    uniq_labels = np.unique(c_labels)

    for label in uniq_labels:
        label_mask = c_labels == label
        ctx = data["context"][label_mask].iloc[0]
        centroids[label] = (np.mean(embeddings[label_mask], axis=0), ctx)
        
    return centroids


def give_result(model, data, clus_alg, emb_alg, norm, train=True):
    """    
        Calculates the context vector based on sum of word embeddings.

        Parameters:
            - model: dictionary like structure, gives embedding by the word: model[word];
            - data: Pandas Dataframe that has necessary columns:
                "embedding_need", "word", "gold_sense_id";
            - clus_alg: clustering function: e.g. 'Kmeans', 'SpectralClustering';
            - emb_alg: algorithm to get context vector;
            - norm: True - normalize, False - do not;
            - train: True - evaluate with ARS, False - do not;
        
        Result: array  
    """

    emb = np.array([emb_alg(model, context, counter, 300, norm) for context, counter in data["embedding_need"]])
    words = data["word"].unique().tolist()
    ars = []
    labels = []
    centroids = {}
    for i, word in enumerate(words):
        mask = data["word"] == word
        clustering = clus_alg.fit(emb[mask])
        labels += clustering.labels_.tolist()
        
        if train:
            ars.append(adjusted_rand_score(clustering.labels_, data.query("word == @word")["gold_sense_id"]))
        
        centroids[word] = get_centroids(emb[mask], clustering.labels_, data.query("word == @word"))

    return np.mean(ars), labels, centroids


def give_2step_result(model, data, clus_alg, emb_alg, norm, pref=-2, train=True):
    """    
        Calculates the context vector based on sum of word embeddings.

        Parameters:
            - model: dictionary like structure, gives embedding by the word: model[word];
            - data: Pandas Dataframe that has necessary columns:
                "embedding_need", "word", "gold_sense_id";
            - clus_alg: clustering function: e.g. 'Kmeans', 'SpectralClustering';
            - emb_alg: algorithm to get context vector;
            - norm: True - normalize, False - do not;
            - pref: preference parameter for AffinityPropagation algorithm;
            - train: True - evaluate with ARS, False - do not;
        
        Result: array  
    """
    clust_func, name_clust_func = clus_alg 
    clus_find_n = AffinityPropagation(preference=pref, random_state=42)
    emb = np.array([emb_alg(model, context, counter, 300, norm) for context, counter in data["embedding_need"]])

    words = data["word"].unique().tolist()
    ars = []
    labels = []
    centroids = {}
    for i, word in enumerate(words):
        parameter = {}
        mask = data["word"] == word
        clustering = clus_find_n.fit(emb[mask])
        parameter["n_clusters"] = len(set(clustering.labels_))
        if name_clust_func != "AgglomerativeClustering":
            parameter["random_state"] = 13
        clustering = clust_func(**parameter).fit(emb[mask])
        labels += clustering.labels_.tolist()
        if train:
            ars.append(adjusted_rand_score(clustering.labels_, data.query("word == @word")["gold_sense_id"]))
        
        centroids[word] = get_centroids(emb[mask], clustering.labels_, data.query("word == @word"))

    return np.mean(ars), labels, centroids

def print_example(model):
    words = list(model.global_dict.keys())
    target_word = words[0]
    context1 = model.global_dict[target_word][0][1]
    context2 = model.global_dict[target_word][1][1]
    print(
        f"Words in the system: {words}\n\nTarget word: {target_word}"
        + f"\n\nFirst context sense : {context1}\n\nSecond context sense : {context2}"
    )

In [6]:
class WSIModel:
    """ Word Sense Induction model based on pretrained word embeddings and clustering"""

    def __init__(self, pretrained_model_path):
        self.model = gensim.models.KeyedVectors.load(pretrained_model_path)
        self.size = self.model.vector_size
        self.rs = 42
        self.embedded = None
        self.params = {
            "is_norm": None,
            "emb_func": None,
            "preference": None,
            "clust_func": None,
        }
        self.best_train_labels = None
        self.test_labels = None       
        self.global_dict = {}

    def _update_params(self, is_norm, emb_func, pref, clust_func):
        self.params["is_norm"] = is_norm
        self.params["emb_func"] = emb_func
        self.params["preference"] = pref
        self.params["clust_func"] = clust_func

    def _print_progess(self, emb_func, norm, clust_func, pref, ars_step, steps):
        print(
            f"Embedding Method: {emb_func}; Is_norm = {norm}; " 
            + f"Clustering = {clust_func} ({steps} step); Preference = {pref}"
            + f"\n***Mean ARS = {ars_step}***\n"
        )
    
    def _update_global_dict(self, data, labels, centroids):
        self.global_dict.update(centroids)


    def fit(self, train_data, nv=(True,), print_all=True, app=(-3, -2, -1)):
        """    
            Fit the model parameters and returns train_data labels.

            Parameters:
                - train_data: Pandas Dataframe that has necessary columns:
                    "embedding_need", "word", "gold_sense_id";
                - nv: normalization parameters True/False ;
                - print_all: print the results of every model to stdout;
                - app: affinity propagation preference:
                    iterable structure that has preferences for AP algorithm;
        
            Result: train_data clustering labels,
                the order cooresponds to rows of train_data  
        """

        embedding_funcs = (give_emb_local_inv, give_emb_average, give_emb_sum)
        names_embedding_funcs = ("Local_Average", "Average", "Sum")

        clustering_funcs = (
            KMeans, AgglomerativeClustering, SpectralClustering, 
        )
        names_clustering_funcs = (
            "KMeans", "AgglomerativeClustering", "SpectralClustering",
        )

        norm_variants = nv
        affinity_propagation_preference = app

        centroids = None
        best_train_labels = None
        ars_init = 0.0
        for is_norm in norm_variants:
            for emb_func, name_emb_func in zip(embedding_funcs, names_embedding_funcs):
                for pref in affinity_propagation_preference:
                    for clust_func, name_clust_func in zip(clustering_funcs, names_clustering_funcs):
                        
                        # Get results for two step algorithm:
                        ars_2step, labels_2step, centroids2s = give_2step_result(
                            self.model,
                            train_data,
                            (clust_func, name_clust_func),
                            emb_func,
                            is_norm,
                            pref,
                        )
                        if print_all:
                            self._print_progess(
                                name_emb_func, is_norm,
                                name_clust_func, pref, ars_2step, 2,
                            )
                        
                        if ars_init <= ars_2step:
                            ars_init = ars_2step
                            best_train_labels = labels_2step
                            # Update parameters of the model:
                            self._update_params(
                                is_norm, emb_func,
                                pref, clust_func,
                            )
                            self._print_progess(
                                name_emb_func, is_norm,
                                name_clust_func, pref, ars_2step, 2,
                            )
                            centroids = centroids2s
                    
                    # Get results for one step algorithm:
                    ars_1step, labels_1step, centroids1s = give_result(
                            self.model,
                            train_data,
                            AffinityPropagation(preference=pref, random_state=self.rs),
                            emb_func,
                            is_norm,
                    )
                    if print_all:
                        self._print_progess(
                            name_emb_func, is_norm,
                            "AffinityPropagation", pref, ars_2step, 1,
                        )

                    if ars_init <= ars_1step:
                        ars_init = ars_1step
                        best_train_labels = labels_1step
                        # Update parameters of the model:
                        self._update_params(
                            is_norm, emb_func,
                            pref, -1,
                        )
                        self._print_progess(
                            name_emb_func, is_norm,
                            "AffinityPropagation", pref, ars_2step, 1,
                        )
                        centroids = centroids1s

        self._update_global_dict(train_data, best_train_labels, centroids)
        self.best_train_labels = best_train_labels
        return best_train_labels


    def predict(self, test_data):
        """    
            Predict the labels for test_data.

            Parameters:
                - test_data: Pandas Dataframe that has necessary columns:
                    "embedding_need", "word";

            Result: test_data clustering labels,
                the order cooresponds to rows of test_data  
        """
        centroids = None
        if self.params["clust_func"] == -1:
            clust_func = AffinityPropagation(
                preference=self.params["preference"],
                random_state=self.rs,
            )
            _, labels, centroids = give_result(
                self.model,
                test_data,
                clust_func,
                self.params["emb_func"],
                self.params["is_norm"],
                train=False
            )
        else:
            _, labels, centroids = give_2step_result(
                self.model,
                test_data,
                (self.params["clust_func"], self.params["clust_func"].__name__),
                self.params["emb_func"],
                self.params["is_norm"],
                self.params["preference"],
                train=False,
            )
        
        self._update_global_dict(test_data, labels, centroids)
        self.test_labels = labels
        return labels


    def get_cluster_label(word, context):
        """Gives label by word and context list"""
        
        default_cluster = 0
        counter = Counter(s)
        cxt = list(set(context))
        context_emb = self.params["emb_func"](self.model, cxt, counter, self.size, self.params["is_norm"])
        
        if word not in self.global_dict.keys():
            return None
        
        candidates = self.global_dict[word]
        result = {clust_label: context_emb.dot() for clust_label, av_emb in candidates.items()}
        return max(result.items(), key=operator.itemgetter(1))[0]

## 3.5 Train and evaluate the models:

In [7]:
cols = ["context_id", "word", "gold_sense_id", "predict_sense_id"]
emb_model_18 = 'ru_fasttext_model/model.model'
emb_model_20 = 'new_model/model.model'

## *wiki-wiki dataset:*

### **ruscorpora_upos_skipgram_300_5_2018:**

In [8]:
# Train/Evaluate/Get test for CodaLab:

wsi_model1 = WSIModel(emb_model_18)
wiki_train_labels1 = wsi_model1.fit(wiki_train, print_all=False)
wiki_test_labels1 = wsi_model1.predict(wiki_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.6919861201365022***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.7508223416134545***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.7851293039268663***

Embedding Method: Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.7957682490549647***

Embedding Method: Sum; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.7957682490549647***



In [9]:
print_example(wsi_model1)

Words in the system: ['замок', 'лук', 'суда', 'бор', 'банка', 'белка', 'бит', 'горе', 'граф', 'душ']

Target word: замок

First context sense : ['владимир', 'мономах', 'любеч', 'многочисленный', 'укреплять', 'монастырь', 'также', 'являться', 'таковой', 'это', 'крепость', 'ранний', 'европейский', 'строиться', 'преимущественно', 'дерево', 'опоясываться', 'деревянный', 'ограда', 'палисад', 'вокруг', 'становиться', 'появляться', 'ров', 'пример', 'мочь', 'служить', 'вышгородский', 'киевский', 'князь', 'каменный', 'замковый', 'строительство', 'распространяться', 'западный', 'центральный', 'европа', 'лишь', 'xii', 'век', 'главный', 'часть', 'средневековый', 'являться', 'центральный', 'башня', 'донжон', 'выполнять', 'функция', 'цитадель', 'помимо', 'свой', 'оборонительный', 'функция', 'донжон', 'являться', 'непосредственный', 'жилище', 'феодал', 'также', 'главный', 'башня']

Second context sense : ['аркарт', 'открывать', 'протяжение', 'весь', 'год', 'часы', 'посещение', 'апрель', 'сентябрь', '

In [10]:
# Save the results:

wiki_test["predict_sense_id"] = wiki_test_labels1
wiki_test[cols].to_csv("wiki.csv", sep='\t', index=None)
!zip results/wiki_wiki_18.zip wiki.csv
!rm wiki.csv

  adding: wiki.csv (deflated 85%)


### **geowac_lemmas_none_fasttextskipgram_300_5_2020:**

In [11]:
# Train/Evaluate/Get test for CodaLab:

wsi_model2 = WSIModel(emb_model_20)
wiki_train_labels2 = wsi_model2.fit(wiki_train, print_all=False)
wiki_test_labels2 = wsi_model2.predict(wiki_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.8189711565185964***

Embedding Method: Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.8193169968725392***

Embedding Method: Sum; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.8193169968725392***



In [12]:
print_example(wsi_model2)

Words in the system: ['замок', 'лук', 'суда', 'бор', 'банка', 'белка', 'бит', 'горе', 'граф', 'душ']

Target word: замок

First context sense : ['владимир', 'мономах', 'любеч', 'многочисленный', 'укреплять', 'монастырь', 'также', 'являться', 'таковой', 'это', 'крепость', 'ранний', 'европейский', 'строиться', 'преимущественно', 'дерево', 'опоясываться', 'деревянный', 'ограда', 'палисад', 'вокруг', 'становиться', 'появляться', 'ров', 'пример', 'мочь', 'служить', 'вышгородский', 'киевский', 'князь', 'каменный', 'замковый', 'строительство', 'распространяться', 'западный', 'центральный', 'европа', 'лишь', 'xii', 'век', 'главный', 'часть', 'средневековый', 'являться', 'центральный', 'башня', 'донжон', 'выполнять', 'функция', 'цитадель', 'помимо', 'свой', 'оборонительный', 'функция', 'донжон', 'являться', 'непосредственный', 'жилище', 'феодал', 'также', 'главный', 'башня']

Second context sense : ['строение', 'калькировать', 'немой', 'slōʒ', 'средневерхненемецкий', 'слово', 'калькировать', 'з

In [13]:
# Save the results:

wiki_test["predict_sense_id"] = wiki_test_labels2
wiki_test[cols].to_csv("wiki.csv", sep='\t', index=None)
!zip results/wiki_wiki_20.zip wiki.csv
!rm wiki.csv

  adding: wiki.csv (deflated 85%)


## *bts-rnc dataset:*

### **ruscorpora_upos_skipgram_300_5_2018:**

In [14]:
# Train/Evaluate/Get test for CodaLab:

wsi_model1 = WSIModel(emb_model_18)
bts_train_labels1 = wsi_model1.fit(bts_train, print_all=False)
bts_test_labels1 = wsi_model1.predict(bts_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.11164311895880123***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.12202106591591551***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.12294224691023192***

Embedding Method: Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -2
***Mean ARS = 0.125985784596124***

Embedding Method: Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.12725871588006926***

Embedding Method: Sum; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.12725871588006926***



In [15]:
print_example(wsi_model1)

Words in the system: ['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка', 'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира', 'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь', 'среда', 'хвост', 'штамп', 'акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика', 'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра', 'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо', 'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук', 'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз', 'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог', 'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка']

Target word: балка

First context sense : ['пантюхин', 'склиф', 'выползать', 'улица', 'успевать', 'золотков', 'обрушиваться', 'душа', 'компания', 'парень', 'летний', 'геннадий']

Second context sense : ['маленький', 'комната', 'очень', 'высокий', 

In [16]:
# Save the results:

bts_test["predict_sense_id"] = bts_test_labels1
bts_test[cols].to_csv("bts.csv", sep='\t', index=None)
!zip results/bts_18.zip bts.csv
!rm bts.csv

  adding: bts.csv (deflated 86%)


### **geowac_lemmas_none_fasttextskipgram_300_5_2020:**

In [17]:
# Train/Evaluate/Get test for CodaLab:

wsi_model2 = WSIModel(emb_model_20)
bts_train_labels2 = wsi_model2.fit(bts_train, print_all=False)
bts_test_labels2 = wsi_model2.predict(bts_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.08478654598201153***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.08840341279182526***

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -2
***Mean ARS = 0.1415032087485018***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.1446191574482564***



In [18]:
print_example(wsi_model2)

Words in the system: ['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка', 'крыло', 'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира', 'мина', 'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 'поток', 'проказа', 'пропасть', 'проспект', 'пытка', 'рысь', 'среда', 'хвост', 'штамп', 'акция', 'баба', 'байка', 'бум', 'бычок', 'вал', 'газ', 'гвоздика', 'гипербола', 'град', 'гусеница', 'дождь', 'домино', 'забой', 'икра', 'кабачок', 'капот', 'карьер', 'кличка', 'ключ', 'кок', 'кольцо', 'концерт', 'котелок', 'крона', 'круп', 'кулак', 'лейка', 'лук', 'мандарин', 'ножка', 'опора', 'патрон', 'печать', 'пол', 'полоз', 'почерк', 'пробка', 'рак', 'рок', 'свет', 'секрет', 'скат', 'слог', 'стан', 'стопка', 'таз', 'такса', 'тюрьма', 'шах', 'шашка']

Target word: балка

First context sense : ['равнозначный', 'обеспечивать', 'меланхоличный', 'езда', 'цельнометаллический', 'тело', 'усиленный', 'передний', 'задний', 'поперечный', 'наряду', 'работа', 'подвеска', 'превращать', 'машина', 'глыба', 'в

In [19]:
# Save the results:

bts_test["predict_sense_id"] = bts_test_labels2
bts_test[cols].to_csv("bts.csv", sep='\t', index=None)
!zip results/bts_20.zip bts.csv
!rm bts.csv

  adding: bts.csv (deflated 86%)


## *active-dict dataset:*

### **ruscorpora_upos_skipgram_300_5_2018:**

In [20]:
# Train/Evaluate/Get test for CodaLab:

wsi_model1 = WSIModel(emb_model_18)
actd_train_labels1 = wsi_model1.fit(actd_train, print_all=False)
actd_test_labels1 = wsi_model1.predict(actd_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.0030594407098070357***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -3
***Mean ARS = 0.0034982330697388288***

Embedding Method: Local_Average; Is_norm = True; Clustering = SpectralClustering (2 step); Preference = -3
***Mean ARS = 0.003774388077124375***

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -2
***Mean ARS = 0.07108699785486196***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.07349557370249891***

Embedding Method: Local_Average; Is_norm = True; Clustering = SpectralClustering (2 step); Preference = -2
***Mean ARS = 0.08018192496429273***

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -1
***Mean ARS = 0.24114998545517347***



In [21]:
print_example(wsi_model1)

Words in the system: ['дар', 'двигатель', 'двойник', 'дворец', 'девятка', 'дедушка', 'дежурная', 'дежурный', 'декабрист', 'декрет', 'дело', 'демобилизация', 'демократ', 'демонстрация', 'дерево', 'держава', 'дерзость', 'десятка', 'десяток', 'деятель', 'диалог', 'диаметр', 'диплом', 'директор', 'диск', 'дичь', 'длина', 'доброволец', 'добыча', 'доказательство', 'доктор', 'долгота', 'доля', 'дом', 'дорога', 'достижение', 'древесина', 'дупло', 'дура', 'дух', 'дым', 'дымка', 'дыхание', 'дьявол', 'евро', 'езда', 'жаворонок', 'жало', 'жертва', 'жестокость', 'жидкость', 'жила', 'жилец', 'жир', 'жребий', 'заведение', 'завещание', 'зависимость', 'заголовок', 'заготовка', 'задание', 'задача', 'задержка', 'зажигалка', 'закон', 'закрытие', 'заложник', 'замена', 'западня', 'запятая', 'застой', 'затея', 'затишье', 'затмение', 'затруднение', 'захоронение', 'звезда', 'звон', 'зеркало', 'зло', 'злоупотребление', 'знак', 'знамя', 'значение', 'зонт', 'давление', 'дама', 'данные', 'дата', 'двойка', 'двор', 

In [22]:
# Save the results:

actd_test["predict_sense_id"] = actd_test_labels1
actd_test[cols].to_csv("actd.csv", sep='\t', index=None)
!zip results/actd_18.zip actd.csv
!rm actd.csv

  adding: actd.csv (deflated 85%)


### **geowac_lemmas_none_fasttextskipgram_300_5_2020:**

In [23]:
# Train/Evaluate/Get test for CodaLab:

wsi_model2 = WSIModel(emb_model_20)
actd_train_labels2 = wsi_model2.fit(actd_train, print_all=False)
actd_test_labels2 = wsi_model2.predict(actd_test)

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -3
***Mean ARS = 0.00996843744857003***

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -2
***Mean ARS = 0.07521435080938825***

Embedding Method: Local_Average; Is_norm = True; Clustering = AgglomerativeClustering (2 step); Preference = -2
***Mean ARS = 0.08899651456516436***

Embedding Method: Local_Average; Is_norm = True; Clustering = SpectralClustering (2 step); Preference = -2
***Mean ARS = 0.09119365037811128***

Embedding Method: Local_Average; Is_norm = True; Clustering = KMeans (2 step); Preference = -1
***Mean ARS = 0.2636972493668461***

Embedding Method: Local_Average; Is_norm = True; Clustering = SpectralClustering (2 step); Preference = -1
***Mean ARS = 0.29456321892257964***

Embedding Method: Average; Is_norm = True; Clustering = SpectralClustering (2 step); Preference = -1
***Mean ARS = 0.30311397841984955***

Embedding Method: Sum; 

In [24]:
print_example(wsi_model2)

Words in the system: ['дар', 'двигатель', 'двойник', 'дворец', 'девятка', 'дедушка', 'дежурная', 'дежурный', 'декабрист', 'декрет', 'дело', 'демобилизация', 'демократ', 'демонстрация', 'дерево', 'держава', 'дерзость', 'десятка', 'десяток', 'деятель', 'диалог', 'диаметр', 'диплом', 'директор', 'диск', 'дичь', 'длина', 'доброволец', 'добыча', 'доказательство', 'доктор', 'долгота', 'доля', 'дом', 'дорога', 'достижение', 'древесина', 'дупло', 'дура', 'дух', 'дым', 'дымка', 'дыхание', 'дьявол', 'евро', 'езда', 'жаворонок', 'жало', 'жертва', 'жестокость', 'жидкость', 'жила', 'жилец', 'жир', 'жребий', 'заведение', 'завещание', 'зависимость', 'заголовок', 'заготовка', 'задание', 'задача', 'задержка', 'зажигалка', 'закон', 'закрытие', 'заложник', 'замена', 'западня', 'запятая', 'застой', 'затея', 'затишье', 'затмение', 'затруднение', 'захоронение', 'звезда', 'звон', 'зеркало', 'зло', 'злоупотребление', 'знак', 'знамя', 'значение', 'зонт', 'давление', 'дама', 'данные', 'дата', 'двойка', 'двор', 

In [25]:
# Save the results:

actd_test["predict_sense_id"] = actd_test_labels2
actd_test[cols].to_csv("actd.csv", sep='\t', index=None)
!zip results/actd_20.zip actd.csv
!rm actd.csv

  adding: actd.csv (deflated 85%)


### extra:

In [1]:
#!rm -fr results
#!rm -fr ru_fasttext_model
#!rm -fr new_model
#!rm -fr russe-wsi-kit
#!rm README 